<a href="https://colab.research.google.com/github/EsquivelJC/hacking-civico/blob/master/tareas/03-exploracion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploración de datos: COVID-19 en México

En este cuaderno aprenderemos algunas bases de exploración de datos. Los datos a utilizar son los datos abiertos sobre COVID-19 en México, los cuales pueden ser accedidos directamente desde el portal: https://www.gob.mx/salud/documentos/datos-abiertos-152127

Del portal de datos abiertos podemos encontrar dos enlaces, uno para descargar el conjunto de datos y otro para descargar el diccionario; así que podemos automatizar el proceso de descarga y descompresión de la información.

Para obtener los datos, sólo basta que ejecutes la siguiente línea de código, que contiene el comando de descarga de la información.

In [1]:
# Para descargar el conjunto de datos
!wget http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip

# Para descargar el diccionario de los datos
!wget http://epidemiologia.salud.gob.mx/gobmx/salud/datos_abiertos/diccionario_datos_covid19.zip

# Descomprimimos ambas carpetas y movemos archivos
!unzip datos_abiertos_covid19.zip
!unzip diccionario_datos_covid19.zip
!mv ./diccionario_datos_covid19/*.xlsx .
!mv *COVID19MEXICO.csv COVID19MEXICO.csv

# Eliminamos las carpetas comprimidas
!rm -rf diccionario_datos_covid19
!rm datos_abiertos_covid19.zip
!rm diccionario_datos_covid19.zip

--2020-09-14 02:21:31--  http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip
Resolving datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)... 187.210.186.146
Connecting to datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)|187.210.186.146|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32082132 (31M) [application/x-zip-compressed]
Saving to: ‘datos_abiertos_covid19.zip’

datos_abiertos_covi 100%[===================>]  30.60M  13.1MB/s    in 2.3s    

2020-09-14 02:21:34 (13.1 MB/s) - ‘datos_abiertos_covid19.zip’ saved [32082132/32082132]

--2020-09-14 02:21:34--  http://epidemiologia.salud.gob.mx/gobmx/salud/datos_abiertos/diccionario_datos_covid19.zip
Resolving epidemiologia.salud.gob.mx (epidemiologia.salud.gob.mx)... 187.191.75.207
Connecting to epidemiologia.salud.gob.mx (epidemiologia.salud.gob.mx)|187.191.75.207|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ep

## Carga

Ahora que tenemos los datos, procederemos a cargarlos. Para cargar los datos, primero importaremos algunos paquetes de Python, pues su funcionalidad nos será muy util.

In [2]:
import statistics

import pandas as pd
import numpy as np

In [3]:
# Especificamos el nombre del archivo por abrir
nombre_archivo = 'COVID19MEXICO.csv'

# Utilizamos la función de Pandas para cargar un archivo CSV
data = pd.read_csv(nombre_archivo, encoding='latin-1')

Ahora procedemos a cargar el archivo con los descriptores.

In [4]:
nombre_diccionario = 'Descriptores_0419.xlsx'
diccionario = pd.read_excel(nombre_diccionario)

Finalmente, cargamos el rachivo de catálogos.

In [5]:
nombre_cat = 'Catalogos_0412.xlsx'
sheets = ['ORIGEN', 'SECTOR', 'SEXO', 'TIPO_PACIENTE', 'SI_NO', 'NACIONALIDAD', 'RESULTADO', 'de ENTIDADES', 'MUNICIPIOS']
catalogues = {}
for sheet in sheets:
  catalogue = pd.read_excel(nombre_cat, sheet_name='Catálogo ' + sheet)
  catalogue.columns = catalogue.columns.str.replace(' ', '_')
  catalogue.columns = catalogue.columns.str.replace('.', '')
  catalogue.columns = catalogue.columns.str.lower()
  new_catalogue = {sheet: catalogue}
  catalogues.update(new_catalogue)

##Perfilamiento

###Descripción General

In [6]:
# limpieza del nombre de las columnas, remover espacios, carácteres especiales y pasar a minúsculas
data.columns = data.columns.str.replace(' ', '_')
data.columns = data.columns.str.replace('.', '')
data.columns = data.columns.str.lower()

###Limpieza

In [7]:
# convertimos a fecha algunas columnas
def cast_datetime_cols(df):
    timestamp_cols = [col for col in df.columns if "fecha" in col]
    df[timestamp_cols] = df[timestamp_cols].apply(lambda date_col: pd.to_datetime(date_col, errors="coerce"), axis=0)
    return df

data = cast_datetime_cols(data)

In [8]:
# unimos los datos con los catalogos para tener mejor entendimiento de ellos
data = (
    # unimos con el catálogo MUNICIPIOS y ENTIDADES
    data.merge(catalogues['de ENTIDADES'], how='left', left_on='entidad_res', right_on='clave_entidad').
    drop(columns=['entidad_res', 'abreviatura']).
    rename(columns={'entidad_federativa':'entidad_res'}).
    merge(catalogues['MUNICIPIOS'], how='left', left_on=['clave_entidad','municipio_res'], right_on=['clave_entidad','clave_municipio']).
    drop(columns=['municipio_res', 'clave_entidad', 'clave_municipio']).
    rename(columns={'municipio':'municipio_res'})
)

In [9]:
# unimos con el catálogo de ENTIDADES
entidades_cols = [col for col in data.columns if "entidad" in col and col != 'entidad_res']
for col in entidades_cols:
    data = (
        data.merge(catalogues['de ENTIDADES'], how='left', left_on=col, right_on='clave_entidad').
        drop(columns=[col, 'abreviatura', 'clave_entidad']).
        rename(columns={'entidad_federativa':col})
    )

In [10]:
# unimos con el catálogo SI_NO
sino_cols = ['intubado', 'neumonia', 'embarazo', 'habla_lengua_indig', 'diabetes', 'epoc', 'asma', 'inmusupr',
             'hipertension', 'otra_com', 'cardiovascular', 'obesidad', 'renal_cronica', 'tabaquismo', 
             'otro_caso', 'migrante', 'uci']
for col in sino_cols:
    data = (
        data.merge(catalogues['SI_NO'], how='left', left_on=col, right_on='clave').
        drop(columns=[col, 'clave']).
        rename(columns={'descripción':col})
    )

> **Nota:**
>
> Si ejecutaste una versión previa del cuaderno, notarás que en la siguiente sección aparecía un error. A continuación dejamos como un pequeño paréntesis un poco más de limpieza sobre el cuaderno.

1. Corregimos el nombre de las columnas:

In [11]:
catalogues['RESULTADO'].columns

Index(['unnamed:_0', 'unnamed:_1'], dtype='object')

In [12]:
catalogues['RESULTADO'].columns = ['clave', 'descripción']
catalogues['RESULTADO'].columns

Index(['clave', 'descripción'], dtype='object')

2. Removemos la (primera) fila que no nos es útil.

In [13]:
catalogues['RESULTADO'] = catalogues['RESULTADO'].drop(index=0)

3. Convertimos el tipo de dato.

In [14]:
catalogues['RESULTADO']['clave'] = catalogues['RESULTADO']['clave'].astype(int)

> Fin del paréntesis. Hasta esta sección, esta parte de limpieza permite continuar con el proceso de manera normal.

In [15]:
# unimos otras columnnas que comparten lógica
other_cols = ['origen', 'sector', 'sexo', 'tipo_paciente', 'nacionalidad', 'resultado']
for col, cat in zip(other_cols, [x.upper() for x in other_cols]):
    data = (
        data.merge(catalogues[cat], how='left', left_on=col, right_on='clave').
        drop(columns=[col, 'clave']).
        rename(columns={'descripción':col})
    )

In [16]:
# creamos algunas variables que podrían ser útiles después
data['defuncion'] = ~data.fecha_def.isna()
data['mes_def'] = data.fecha_def.dt.month
data['año_def'] = data.fecha_def.dt.year
data['mes_ingreso'] = data.fecha_ingreso.dt.month
data['año_ingreso'] = data.fecha_ingreso.dt.year

# **Tarea 3**

# ¿Tienen los pacientes con hipertensión un riesgo más alto de defunción?
Al parecer, es un .04% de riesgo si no tiene hipertensión y .16% de riesgo si sí tiene hipertensión, por lo que estos pacientes tienen un aumento considerable del .08% de riesgo dada su enfermedad.

In [17]:
data.groupby('hipertension').defuncion.value_counts(normalize=True)

hipertension  defuncion
NO            False        0.960811
              True         0.039189
SE IGNORA     False        0.872118
              True         0.127882
SI            False        0.834710
              True         0.165290
Name: defuncion, dtype: float64

#¿Cuántos casos confirmados se tienen por Estado?

In [25]:
data[data.resultado == 'Positivo SARS-CoV-2'].entidad_res.value_counts().sort_index()

resultado            entidad_res                    
Positivo SARS-CoV-2  AGUASCALIENTES                       6392
                     BAJA CALIFORNIA                     18328
                     BAJA CALIFORNIA SUR                  8946
                     CAMPECHE                             5795
                     CHIAPAS                              6380
                     CHIHUAHUA                            8981
                     CIUDAD DE MÉXICO                   111443
                     COAHUILA DE ZARAGOZA                24043
                     COLIMA                               4188
                     DURANGO                              7723
                     GUANAJUATO                          36646
                     GUERRERO                            16454
                     HIDALGO                             11436
                     JALISCO                             23221
                     MICHOACÁN DE OCAMPO                 17614
  

#¿Cuántas defunciones se tienen por Estado?

In [32]:
data[data.defuncion == True].entidad_res.value_counts().sort_index()

AGUASCALIENTES                       605
BAJA CALIFORNIA                     4085
BAJA CALIFORNIA SUR                  541
CAMPECHE                             959
CHIAPAS                             1221
CHIHUAHUA                           2043
CIUDAD DE MÉXICO                   12038
COAHUILA DE ZARAGOZA                2215
COLIMA                               603
DURANGO                              721
GUANAJUATO                          3222
GUERRERO                            2307
HIDALGO                             2095
JALISCO                             3918
MICHOACÁN DE OCAMPO                 1937
MORELOS                             1256
MÉXICO                             13682
NAYARIT                              826
NUEVO LEÓN                          3558
OAXACA                              1624
PUEBLA                              4636
QUERÉTARO                           1014
QUINTANA ROO                        1927
SAN LUIS POTOSÍ                     1790
SINALOA         

#¿Cuántos fallecimientos han ocurrido en el Estado con mayor número de casos confirmados?
La ciudad de México es quien tiene mayor número de casos confirmados con 12038

#¿Cuántos fallecimientos han ocurrido en los pacientes Ambulatorios?
Han fallecido 9633 en los pacientes Ambulatorios

In [28]:
data.groupby('tipo_paciente').defuncion.value_counts()

tipo_paciente  defuncion
AMBULATORIO    False        1244673
               True            9633
HOSPITALIZADO  False         181464
               True           80818
Name: defuncion, dtype: int64